## <a href='https://leetcode.com/problems/number-of-longest-increasing-subsequence/'>673. Number of Longest Increasing Subsequence</a>
* https://leetcode.com/articles/a-recursive-approach-to-segment-trees-range-sum-queries-lazy-propagation/
* https://cp-algorithms.com/sequences/longest_increasing_subsequence.html#toc-tgt-9
* https://www.geeksforgeeks.org/longest-monotonically-increasing-subsequence-size-n-log-n/

> “Tell me and I will forget. Show me and I will remember. Involve me and I will understand.”

Given an integer array nums, return the number of longest increasing subsequences.

__Example 1:__
```
Input: nums = [1,3,5,4,7]
Output: 2
Explanation: The two longest increasing subsequences are [1, 3, 4, 7] and [1, 3, 5, 7].
```
__Example 2:__
```
Input: nums = [2,2,2,2,2]
Output: 5
Explanation: The length of longest continuous increasing subsequence is 1, and there are 5 subsequences' length is 1, so output 5.
```
__Constraints:__

* 1 <= nums.length <= 2000
* -106 <= nums[i] <= 106

In [6]:
public class SolutionRec {
    private int answer = 0, nums[], max = 0, n = 0;
    private Map<Integer, Integer> map = new HashMap<>();
    public int solve(int[] nums) {
        this.nums = nums;
        n = nums.length;
        helper(0, -1, 1);
        return answer;
    }
    private void helper(int start, int prev, int length) {
        if (start >= n) {
            if (length > max) {
                max = length;
                answer = 0;
            }
            if (length == max) answer++;
            return;
        }
        if (prev == -1 || nums[prev] < nums[start]) helper(start + 1, start, length + 1);
        helper(start + 1, prev, length);
    }
}

In [7]:
new SolutionRec().solve(new int[] {2,2,2,2,2});

5

In [8]:
new SolutionRec().solve(new int[] {1,3,2,4,5});

2

In [17]:
public class SolutionDp {
    public int solve(int[] nums) {
        int n = nums.length;
        int[] lengths = new int[n];
        int[] counts = new int[n];
        Arrays.fill(lengths, 1);
        Arrays.fill(counts, 1);
        //     *
        // 1 3 2 4 5
        // ^   
        // 1 1 1 1 1
        for (int j = 0; j < n; ++j) {
            for (int i = 0; i < j; ++i) {
                if (nums[i] >= nums[j]) continue;
                if (lengths[i] >= lengths[j]) {
                    lengths[j] = lengths[i] + 1;
                    counts[j] = counts[i];
                } else if (lengths[i] + 1 == lengths[j]) {
                    counts[j] += counts[i];
                }
            }
        }
        int longest = 0, answer = 0;
        for (int length : lengths) longest = Math.max(length, longest);
        for (int i = 0; i < n; ++i) {
            if (lengths[i] == longest) answer += counts[i];
        }
        return answer;
    }
}

In [18]:
new SolutionDp().solve(new int[] {2,2,2,2,2});

5

In [19]:
new SolutionDp().solve(new int[] {1,3,2,4,5});

2

In [20]:
new SolutionDp().solve(new int[] {1,3,5,4,7});

2

In [27]:
public class SolutionSegTree {
    public int solve(int[] nums) {
        int n = nums.length;
        int min = nums[0], max = nums[0];
        for (int num : nums) {
            min = Math.min(num, min);
            max = Math.max(num, max);
        }
        Node root = new Node(min, max);
        for (int num : nums) {
            Value val = query(root, num - 1);
            insert(root, num, new Value(val.len + 1, val.count));
        }
        return root.val.count;
    }
    private Value query(Node node, int key) {
        if (node.rangeRight <= key) return node.val;
        else if (node.rangeLeft > key) return new Value(0, 1);
        else return merge(query(node.getLeft(), key), query(node.getRight(), key));
    }
    private void insert(Node node, int key, Value val) {
        if (node.rangeLeft == node.rangeRight) {
            node.val = merge(val, node.val);
        } else {
            if (key <= node.getMid()) {
                insert(node.getLeft(), key, val);
            } else {
                insert(node.getRight(), key, val);
            }
            node.val = merge(node.getLeft().val, node.getRight().val);
        }
    }
    private Value merge(Value v1, Value v2) {
        if (v1.len == v2.len) {
            if (v1.len == 0) return new Value(0, 1);
            return new Value(v1.len, v1.count + v2.count);
        }
        return v1.len > v2.len ? v1 : v2;
    }
    private class Node {
        public int rangeLeft, rangeRight;
        public Node left, right;
        public Value val = new Value(0, 1);
        public Node (int rangeLeft, int rangeRight) {
            this.rangeLeft = rangeLeft;
            this.rangeRight = rangeRight;
        }
        public int getMid() {
            return rangeLeft + (rangeRight - rangeLeft) / 2;
        }
        public Node getLeft() {
            if (left == null) left = new Node(rangeLeft, getMid());
            return left;
        }
        public Node getRight() {
            if (right == null) right = new Node(getMid() + 1, rangeRight);
            return right;
        }
        public void printLeaves() {
            LinkedList<Node> queue = new LinkedList<>();
            queue.offer(this);
            while (!queue.isEmpty()) {
                Node cur = queue.poll();
                if (cur.left == null && cur.right == null) {
                    System.out.println(cur);
                } else {
                    if (cur.left != null) queue.offer(cur.left);
                    if (cur.right != null) queue.offer(cur.right);
                }
            }
        }        
    }
    private class Value {
        public int len, count;
        public Value(int len, int count) {
            this.len = len;
            this.count = count;
        }
    }
}

In [28]:
new SolutionSegTree().solve(new int[] {1,3,2,4,5});

2

In [29]:
new SolutionSegTree().solve(new int[] {2,2,2,2,2});

5

In [128]:
public class SegmentTreeQueryMinBasedOnArray {
    private int[] tree, nums;
    private int n;
    public void constructTree(int[] nums) {
        this.nums = nums;
        n = nums.length;
        tree = new int[n * 2 + 1];
        constructTreeHelper(0, n - 1, 0);
        System.out.println(Arrays.toString(tree));
    }
    public int rangeMinQuery(int qlo, int qhi) {
        return rangeMinQueryHelper(qlo, qhi, 0, n - 1, 0);
    }
    private int rangeMinQueryHelper(int qlo, int qhi, int lo, int hi, int pos) {
        if (qlo <= lo && qhi >= hi) return tree[pos];     // total overlap  
        if (qlo > hi || qhi < lo) return Integer.MAX_VALUE; // partial overlap
        int mid = lo + (hi - lo) / 2;
        return Math.min(
            rangeMinQueryHelper(qlo, qhi, lo, mid, 2 * pos + 1),
            rangeMinQueryHelper(qlo, qhi, mid + 1, hi, 2 * pos + 2)
        );
    }
    private void constructTreeHelper(int lo, int hi, int pos) {
        System.out.println(String.format("lo=%d hi=%d pos=%d", lo, hi, pos));
        if (lo == hi) {
            tree[pos] = nums[lo];
            return;
        }
        int mid = lo + (hi - lo) / 2;
        constructTreeHelper(lo, mid, 2 * pos + 1);
        constructTreeHelper(mid + 1, hi, 2 * pos + 2);
        tree[pos] = Math.min(tree[2 * pos + 1], tree[2 * pos + 2]);
    }
}

In [129]:
int[] nums = {1,-2,4,5,6,0};
SegmentTreeQueryMinBasedOnArray query = new SegmentTreeQueryMinBasedOnArray();
query.constructTree(nums);

lo=0 hi=5 pos=0
lo=0 hi=2 pos=1
lo=0 hi=1 pos=3
lo=0 hi=0 pos=7
lo=1 hi=1 pos=8
lo=2 hi=2 pos=4
lo=3 hi=5 pos=2
lo=3 hi=4 pos=5
lo=3 hi=3 pos=11
lo=4 hi=4 pos=12
lo=5 hi=5 pos=6
[-2, -2, 0, -2, 4, 5, 0, 1, -2, 0, 0, 5, 6]


In [130]:
query.rangeMinQuery(0, 1);

-2

In [123]:
query.rangeMinQuery(2, 5);

0

In [207]:
int[] nums = {-11,2,-10,100};
SegmentTreeQueryMinBasedOnArray query = new SegmentTreeQueryMinBasedOnArray();
query.constructTree(nums);

lo=0 hi=3 pos=0
lo=0 hi=1 pos=1
lo=0 hi=0 pos=3
lo=1 hi=1 pos=4
lo=2 hi=3 pos=2
lo=2 hi=2 pos=5
lo=3 hi=3 pos=6
[-11, -11, -10, -11, 2, -10, 100, 0, 0]


In [209]:
query.rangeMinQuery(3,3)

100

In [228]:
public class SegmentTreeQueryMinBasedOnNodes {
    private Node root;
    public void constructTree(int[] nums) {
        int n = nums.length;
        root = new Node(0, n - 1);
        for (int i = 0; i < n; ++i) insert(root, i, nums[i]);
        System.out.println(root);
    }
    public int query(int qlo,  int qhi) {
        return queryHelper(root, qlo, qhi);
    }
    private int queryHelper(Node node, int qlo, int qhi) {
        System.out.println(node + String.format("qlo:%d qhi:%d", qlo, qhi));
        if (node.rangeLeft >= qlo && node.rangeRight <= qhi) return node.val;
        if (node.rangeLeft > qhi || node.rangeRight < qlo) return Integer.MAX_VALUE;
        return Math.min(
            queryHelper(node.getLeft(), qlo, qhi),
            queryHelper(node.getRight(), qlo, qhi)
        );
    }
    private void insert(Node node, int key, int val) {
        if (node.rangeLeft == node.rangeRight) {
            node.val = val;
        } else {
            if (key <= node.getMid()) {
                insert(node.getLeft(), key, val);
            } else {
                insert(node.getRight(), key, val);
            }
            node.val = Math.min(node.getLeft().val, node.getRight().val);
        }
    }
    private class Node {
        public int rangeLeft, rangeRight;
        public Node left, right;
        public int val;
        public Node(int rangeLeft, int rangeRight) {
            this.rangeLeft = rangeLeft;
            this.rangeRight = rangeRight;
        }
        public int getMid() {
            return rangeLeft + (rangeRight - rangeLeft) / 2;
        }
        public Node getLeft() {
            if (left == null) left = new Node(rangeLeft, getMid());
            return left;
        }
        public Node getRight() {
            if (right == null) right = new Node(getMid() + 1, rangeRight);
            return right;
        }
        @Override
        public String toString() {
            LinkedList<Node> queue = new LinkedList<>();
            queue.offer(this);
            StringBuilder result = new StringBuilder();
            while (!queue.isEmpty()) {
                int size = queue.size();
                for (int i = 0; i < size; ++i) {
                    Node cur = queue.poll();
                    result.append(
                        String.format("[val=%d(%d-%d)]", cur.val, 
                                      cur.rangeLeft, cur.rangeRight)
                    );
                    if (cur.left != null) queue.offer(cur.left);
                    if (cur.right != null) queue.offer(cur.right);
                }
                result.append('\n');
            }
            return result.toString();
        }
    }
}

In [232]:
SegmentTreeQueryMinBasedOnNodes st = new SegmentTreeQueryMinBasedOnNodes();
st.constructTree(new int[] {-11,2,-10,100});

[val=-11(0-3)]
[val=-11(0-1)][val=-10(2-3)]
[val=-11(0-0)][val=2(1-1)][val=-10(2-2)][val=100(3-3)]



In [233]:
st.query(0, 1);

[val=-11(0-3)]
[val=-11(0-1)][val=-10(2-3)]
[val=-11(0-0)][val=2(1-1)][val=-10(2-2)][val=100(3-3)]
qlo:0 qhi:1
[val=-11(0-1)]
[val=-11(0-0)][val=2(1-1)]
qlo:0 qhi:1
[val=-10(2-3)]
[val=-10(2-2)][val=100(3-3)]
qlo:0 qhi:1


-11

In [238]:
st.query(3, 3)

[val=-11(0-3)]
[val=-11(0-1)][val=-10(2-3)]
[val=-11(0-0)][val=2(1-1)][val=-10(2-2)][val=100(3-3)]
qlo:3 qhi:3
[val=-11(0-1)]
[val=-11(0-0)][val=2(1-1)]
qlo:3 qhi:3
[val=-10(2-3)]
[val=-10(2-2)][val=100(3-3)]
qlo:3 qhi:3
[val=-10(2-2)]
qlo:3 qhi:3
[val=100(3-3)]
qlo:3 qhi:3


100

In [ ]:
// 1 2 5 4 7 8
// 1 2 3 3 4 5
// 1 1 1 1 2 2 

In [247]:
public class MyDpSolution {
    // count total of longest sorted subsets
    public int solve(int[] nums) {
        int n = nums.length;
        int[] lengths = new int[n];
        int[] counts = new int[n];
        Arrays.fill(lengths, 1);
        Arrays.fill(counts, 1);
        for (int j = 0; j < n; ++j) {
            for (int i = 0; i < j; ++i) {
                if (nums[i] >= nums[j]) continue;
                if (lengths[j] <= lengths[i]) {
                    lengths[j] = lengths[i] + 1;
                    counts[j] = counts[i];
                } else if (lengths[i] + 1 == lengths[j]) {
                    counts[j] += counts[i];
                }
            }
        }
        System.out.println(Arrays.toString(lengths));
        System.out.println(Arrays.toString(counts));
        int maxLen = 0;
        int result = 0;
        for (int len : lengths) maxLen = Math.max(len, maxLen);
        for (int i = 0; i < n; ++i) {
            if (lengths[i] == maxLen) result += counts[i]; 
        }
        return result;
    }
}

In [248]:
new MyDpSolution().solve(new int[] {1,2,5,4,7,8});

[1, 2, 3, 3, 4, 5]
[1, 1, 1, 1, 2, 2]


2

In [249]:
new MyDpSolution().solve(new int[] {2,2,2,2,2,2});

[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]


6

In [250]:
new MyDpSolution().solve(new int[] {1,2,3,1,2,3,1,2,3});

[1, 2, 3, 1, 2, 3, 1, 2, 3]
[1, 1, 1, 1, 2, 3, 1, 3, 6]


10